# _prediction()_ function and the PredictionTable Class

The prediction module within SORA was created to predict stellar occultation, organize the prediction and plot the occultation maps. The documentation <a href=https://sora.readthedocs.io/>here</a> contains the details about every step.  

This Jupyter-Notebook was designed as a tutorial for how to work with the prediction module. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.

**All functions have their Docstring containing its main purpose and the needed parameters (physical description and formats). Please, do not hesitate to use it.**

## 0. Index

1. [Predicting stellar occultation](#section_1)
2. [Setting and/or modifying parameters](#section_2)
3. [Apparent Sidereal Time](#section_3)
4. [Ksi and Eta projection](#section_4)
5. [MPC observatories](#section_5)

<a id="section_1"></a>

## 1. Predicting stellar occultation

To predict a stellar occultation, the function `prediction()` must be used. It will need a `Body` Object and the time interval for the search. The function will generate the ephemeris within the given interval (with a default step of 60 seconds between positions, that may be changed by the user) and download the star coordinates from VizieR. The function is able to download stars from Gaia-DR2 and Gaia-EDR3. The user must select the one desired by the keys _"gaiadr2"_ and _"gaiaedr3"_ (default in this version). For every star, it checks the closest ephemeris. An occultation is identified if the distance is smaller than the radius of the Earth plus the radius of the object (given in `Body`) plus the error of the ephemeris (multiplied by a sigma factor given by the user). In the next step, the coordinate of the star is propagated to date using proper motion and parallax. It results in the occultation parameters, such as the Closest Approach distance, the Position Angle, Velocity, etc.

The function returns a `PredictionTable` with all the information.

In [1]:
from sora.prediction import prediction

SORA version: 1.0dev


In [2]:
prediction?

Signature:
prediction(
    time_beg,
    time_end,
    body=None,
    ephem=None,
    mag_lim=None,
    catalogue='gaiadr3',
    step=60,
    divs=1,
    sigma=1,
    radius=None,
    verbose=True,
    reference_center='geocenter',
)
Docstring:
Predicts stellar occultations.

Parameters
----------
time_beg : `str`, `astropy.time.Time`, required
    Initial time for prediction.

time_end : `str`, `astropy.time.Time`, required
    Final time for prediction.

body : `sora.Body`, `str`, default=None
    Object that will occult the stars. It must be a Body object or its name
    to search in the Small Body Database.

ephem : `sora.Ephem`, default=None
    object ephemeris. It must be an Ephemeris object.
    If using a EphemHorizons object, please use 'divs' to make division
    at most a month, or a timeout error may be raised by the Horizon query.

mag_lim : `int`, `float`, `dict`, default=None
    Faintest Gmag allowed in the search.

catalogue : `str`, `VizierCatalogue`
    The catalogu

**Importing and defining the** `Body` **to be used.**

As showed in the docstring of the `prediction()` function, the user can give a `Body` object, an `Ephem` object, or both (given their restrictions). Please refer to the `Body` tutorial for further explanation.

In [3]:
from sora.body import Body
from sora.ephem import EphemKernel

chariklo = Body(name='Chariklo')
ephem = EphemKernel(spkid='2010199', kernels=['input/bsp/Chariklo.bsp', 'input/bsp/de438_small.bsp'], error_ra=0.01, error_dec=0.01)
# here I'm using Body and Ephem so I'm able to give the errors in RA and DEC.
chariklo.ephem = ephem

Obtaining data for Chariklo from SBDB


In [4]:
pred = prediction(body=chariklo, time_beg='2017-06-20 00:00:00', time_end='2017-07-01 00:00:00', mag_lim=18, divs=3)

Ephemeris was split in 3 parts for better search of stars

Searching occultations in part 1/3
Generating Ephemeris between 2017-06-20 00:00:00.000 and 2017-06-23 15:59:00.000 ...
    15 GaiaDR3 stars downloaded
Identifying occultations ...

Searching occultations in part 2/3
Generating Ephemeris between 2017-06-23 16:00:00.000 and 2017-06-27 07:59:00.000 ...
    12 GaiaDR3 stars downloaded
Identifying occultations ...

Searching occultations in part 3/3
Generating Ephemeris between 2017-06-27 08:00:00.000 and 2017-06-30 23:59:00.000 ...
    18 GaiaDR3 stars downloaded
Identifying occultations ...

8 occultations found.


**Important**

The default prediction is made considering the geocenter as center of reference. However, the prediction can also be made for a specific observer. In this case, it is important to note that the search considers only the size of the object and ephemeris error. If the object is very small, consider using a time `step` for initial prediction smaller, or using a different `radius` to avoid losing a promising event. Notice the occultation parameters (closest appoach time and distance, ...) are referred to the observer.

In [5]:
from sora import Observer
outeniqua = Observer(name='Outeniqua', lon='+16 49 17.710', lat='-21 17 58.170', height=1416,
                     ephem=['input/bsp/Chariklo.bsp', 'input/bsp/de438_small.bsp'])

obs_pred = prediction(body=chariklo, time_beg='2017-06-20 00:00:00', time_end='2017-07-01 00:00:00', step=10,
                      mag_lim=18, divs=3, reference_center=outeniqua)

Ephemeris was split in 3 parts for better search of stars

Searching occultations in part 1/3
Generating Ephemeris between 2017-06-20 00:00:00.000 and 2017-06-23 15:59:50.000 ...
    10 GaiaDR3 stars downloaded
Identifying occultations ...

Searching occultations in part 2/3
Generating Ephemeris between 2017-06-23 16:00:00.000 and 2017-06-27 07:59:50.000 ...
    7 GaiaDR3 stars downloaded
Identifying occultations ...

Searching occultations in part 3/3
Generating Ephemeris between 2017-06-27 08:00:00.000 and 2017-06-30 23:59:50.000 ...
    15 GaiaDR3 stars downloaded
Identifying occultations ...

1 occultations found.


In [6]:
obs_pred[0]

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str19
2017-06-22 21:21:25.600,18 55 15.65251 -31 31 21.67062,18 55 15.65251 -31 31 21.67688,0.006,179.52,-22.36,14.659,14.224,52,00:50,149,166,6760223758801661440


In [7]:
pred['2017-06-22 21']

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,long,loct,M-G-T,S-G-T,GaiaDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,str5,float64,float64,str19
2017-06-22 21:18:48.200,18 55 15.65251 -31 31 21.67062,18 55 15.65249 -31 31 21.62190,0.049,359.72,-22.00,14.659,14.224,53,00:50,149,166,6760223758801661440


<a id="section_2"></a>

## 2. The PredictionTable

The resulting variable of the prediction function is a `PredictionTable` Object. The `PredictionTable` Class was created to have all the information about the occultations predicted, plot occultation maps and save the table in other formats.

**The** `PredictionTable` **is a class that inherits all methods from** `Astropy.table.Table` **. All the functions present in** `Astropy.table.Table` **should work for** `PredictionTable`.

More information about it at https://docs.astropy.org/en/stable/table/

**if the user asks to print the prediction, a summary of the table is shown.**

In [8]:
print(pred)

         Epoch             ICRS Star Coord at Epoch    ...  GaiaDR3 Source ID 
                                                       ...                    
----------------------- ------------------------------ ... -------------------
2017-06-21 09:57:43.440 18 55 36.17454 -31 31 19.03261 ... 6760228702284187264
2017-06-22 02:58:37.480 18 55 26.31652 -31 31 20.38440 ... 6760228839723992320
2017-06-22 21:18:48.200 18 55 15.65251 -31 31 21.67062 ... 6760223758801661440
2017-06-23 21:34:37.660 18 55 01.48119 -31 31 22.44257 ... 6760223513963694208
2017-06-24 10:13:39.980 18 54 54.06755 -31 31 22.36732 ... 6760226503261782656
2017-06-25 10:55:09.320 18 54 39.55296 -31 31 21.69987 ... 6760225163236852864
2017-06-26 08:14:35.540 18 54 26.97503 -31 31 20.51255 ... 6760226060885482624
2017-06-26 20:46:07.140 18 54 19.56971 -31 31 20.43382 ... 6760225712991422208


**to print the complete table, we must use:**

In [9]:
pred.pprint_all()

         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    G    long  loct M-G-T S-G-T  GaiaDR3 Source ID 
                                                                                      arcsec  deg   km / s    AU    mag   deg  hh:mm  deg   deg                     
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ---- ----- ----- ----- -------------------
2017-06-21 09:57:43.440 18 55 36.17454 -31 31 19.03261 18 55 36.17500 -31 31 19.60516  0.573 179.41 -21.84  14.663 15.254  225 00:56   128   165 6760228702284187264
2017-06-22 02:58:37.480 18 55 26.31652 -31 31 20.38440 18 55 26.31674 -31 31 20.74213  0.358 179.55 -21.92  14.661 17.929  329 00:53   138   166 6760228839723992320
2017-06-22 21:18:48.200 18 55 15.65251 -31 31 21.67062 18 55 15.65249 -31 31 21.62190  0.049 359.72 -22.00  14.659 14.224   53 00:50   149   166 6760223758801661440
2017-06-23

*M-G-T stands for Moon-Geocenter-Target, the on-sky angle between the star and the moon*

*S-G-T stands for Sun-Geocenter-Target, the on-sky angle between the star and the sun*

**The** `PredictionTable` **can be exported to PRAIA format file**

In this case, some information, such as the Gaia-DR2 Source ID, S-G-T and M-G-T, are lost.

In [10]:
pred.to_praia?

Signature: pred.to_praia(filename)
Docstring:
Writes PredictionTable to PRAIA format.

Parameters
----------
filename : `str`
    Name of the file to save table.
File:      ~/miniconda3/envs/sora-develop-stats/lib/python3.9/site-packages/sora_astro-1.0.dev0-py3.9.egg/sora/prediction/table.py
Type:      method


In [11]:
pred.to_praia('output/Chariklo_occs.table')
# Check your folder named output

**The** `PredictionTable` **can also generate the files in the Occult Watcher feed format**

These are two files, named 'LOG.dat' and 'tableOccult_update.txt'. The occult watcher designation for the object must be informed. For instance, for the satellite Himalia, the designation is "P5M06". If `mode='append'` the prediction will be appended to an existing file. If `mode='restart'`, the file will be overwritted.

In [12]:
pred.to_ow?

Signature: pred.to_ow(ow_des, mode='append')
Docstring:
Writes PredictionTable to OccultWatcher feeder update file format.
Tables will be saved in two files: "tableOccult_update.txt" and "LOG.dat"

Parameters
----------
ow_des : `str`
    Occult Watcher designation for the object.

mode : `str`, default='append'
    Use ``'append'`` to append table to already existing file and
    ``'restart'`` to overwrite existing file.
File:      ~/miniconda3/envs/sora-develop-stats/lib/python3.9/site-packages/sora_astro-1.0.dev0-py3.9.egg/sora/prediction/table.py
Type:      method


In [13]:
pred.to_ow('1997CU26')

**Using the functions from** `Astropy.table.Table` **, the data can be exported to many other formats using the funtion 'write'**

The formats allowed can be seen in https://docs.astropy.org/en/stable/io/unified.html#built-in-readers-writers. This includes latex and csv.

In [14]:
pred.write('output/prediction.tex', overwrite=True)
# Check your folder named output

<a id="section_3"></a>

## 3. Using PredictionTable

The `PredictionTable` was not created to be instantiated directly by the user. It was designed to be an insteresting output object from the prediction function. However, a `PredictionTable` can be instantiated from a PRAIA occultation table. For this, a `PredictionTable` method must be called directly as shown below.

In [15]:
from sora.prediction import PredictionTable

In [16]:
PredictionTable.from_praia?

Signature: PredictionTable.from_praia(filename, name, **kwargs)
Docstring:
Creates a PredictionTable Table reading from a PRAIA table.

Parameters
----------
filename : `str`
    Path to the PRAIA table file.

name : `str`
    Name of the Object of the prediction.

radius : `int`, `float`, optional
    Object radius, in km.
    If not given it's searched in online database.
    When not found online, the default is set to zero.

Returns
-------
 : `sora.prediction.PredictionTable`
    A PredictionTable object.
File:      ~/miniconda3/envs/sora-develop-stats/lib/python3.9/site-packages/sora_astro-1.0.dev0-py3.9.egg/sora/prediction/table.py
Type:      method


In [17]:
pred_1 = PredictionTable.from_praia(filename='output/Chariklo_occs.table', name='Chariklo')

Obtaining data for chariklo from SBDB


AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [ ]:
pred_1.pprint_all()

To remove some occultations the User can use two functions. The first is the _PredictionTable.remove_occ()_.

In [ ]:
pred_1.remove_occ?

In [ ]:
pred_1.remove_occ('2017-06-21 09:57')

In [ ]:
pred_1.pprint_all()

The second function is the `PredictionTable.keep_from_selected_images()` this function will check the names in the saved images and eliminate the occultations without the map. This functions allows that after the user deleted the maps of the unwanted occultation this function will eliminate the respective rows in the `PredictionTable`.  

In [ ]:
pred_1.keep_from_selected_images?

<a id="section_4"></a>

## 4. Plotting occultation maps

SORA is also able to generate an occultation map. The only required parameters are the occultation parameters. The function also has many inputs to configure the plot.

The first time the function is called, cartopy will download some data referring to the features presented in the map, such as the country and state division, lakes, rivers, etc.

In [ ]:
from sora.prediction import plot_occ_map

In [ ]:
plot_occ_map?

**To make it easier to plot the predictions, the `PredictionTable.plot_occ_map()` function can be called which automatically fills the required parameters.**

To plot the map for all the predictions


In [ ]:
pred.plot_occ_map(path='output/')

To plot the map for only one prediction, just give a item to the `PredictionTable` object.

In [ ]:
pred[0].plot_occ_map(path='output/')

**The** `PredictionTable` **can also be plotted by giving the date of the occultation. All the occultations that matches the date will be plotted.**

The date can be as constrained as the user wants, and must match the text that appears in the _'Epoch'_ column.

In [ ]:
pred['2017-06-26'].plot_occ_map()

<a id="section_5"></a>

## 5. Occultation parameters

Finally, a function is implemented in the prediction module which calculates the occultation parameters. For this function, it must be passed a `Star` Object, an `Ephem` Object (it can be any of the `Ephem` classes) and a time. The time does not need to be precise, but it must be close within 60 minutes from the Closest Approach time.

In [ ]:
from sora.prediction import occ_params

In [ ]:
occ_params?

In [ ]:
from sora.star import Star

s = Star(code='6760225712991422208', verbose=False)

In [ ]:
tca, ca, pa, vel, dist = occ_params(star=s, ephem=chariklo.ephem, time='2017-06-26 20:40')

print('Time of the CA:   {} UTC'.format(tca))
print('Closest Approach: {:.3f}'.format(ca))
print('Position Angle:   {:.3f}'.format(pa))
print('Shadow Velocity:  {:.3f}'.format(vel))
print('Object Distance:  {:.5f}'.format(dist))

**This Jupyter-Notebook was designed as a tutorial for how to work with the** `prediction()` **function and** `PredictionTable` **Class. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.**

**The End**